In [ ]:
#Dependencies
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
#import pyLDAvis.gensim #LDA visualization library

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain
import nltk
nltk.download('stopwords')
obj = set(stopwords.words('english'))
print(obj)

{"you've", "you're", 'weren', 'against', 'why', 'but', 't', 'shouldn', 'be', 'below', 'didn', "hasn't", 'wasn', 'were', 'again', 'about', 'and', "mightn't", 'don', 'him', 'is', 'doing', 'from', 'should', 'haven', 'not', 'of', 'y', 'no', "didn't", "it's", "shan't", 'or', "that'll", 'few', 'too', "mustn't", 'over', 'because', 'their', 'after', 'couldn', 'so', 'did', 'more', 'mustn', 'off', 'theirs', 'are', 'between', 'now', 'yours', 'through', 'the', 'll', 'have', 'me', 'who', 'that', 'by', 'into', 'most', 'above', 'only', 'ain', 'isn', 'where', 'has', 'shan', "wasn't", 'being', 'as', "doesn't", 'then', 'such', 'hasn', 'hers', 'i', 'whom', 'having', 'our', 'your', 'had', 'yourself', 'themselves', 'at', 'yourselves', 'been', 'can', 're', 'her', 'an', 'with', "hadn't", 'down', 'will', 'further', "isn't", "weren't", 'was', 'this', 'these', 'herself', 'very', "don't", 'it', 'than', 'she', 'when', 'other', "haven't", 'until', 'aren', 've', 'does', 'under', 'd', 'before', 'in', 'ourselves', "y

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import csv
cw = csv.writer(open("stops.csv",'w'))
cw.writecol(list(obj))

AttributeError: ignored

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ML Datasets/ESG_News.csv')
print(df.shape)
df

(1481, 2)


,topic,text
0,1,"BERLIN, July 19 - Germany's largest consumer ..."
1,2,"LONDON, July 19 - Solid-state batteries could..."
2,3,"STOCKHOLM/COPENHAGEN/OSLO, July 18 - SAS (SAS..."
3,4,"WASHINGTON, July 18 - U.S. allies appear comm..."
4,5,"FARNBOROUGH, England, July 18 - European airp..."
...,...,...
1476,1477,Dec 20 - The United States needs to do a bett...
1477,1478,"BRUSSELS, Dec 20 - The European Commission pl..."
1478,1479,Dec 20 - Morgan Stanley (MS.N) on Monday made...
1479,1480,"BRUSSELS, Dec 20 - A European Union plan to c..."


In [ ]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
df['text_clean']=df['text'].apply(clean)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df

,topic,text,text_clean
0,1,"BERLIN, July 19 - Germany's largest consumer ...","[berlin, july, 19, germany, largest, consumer,..."
1,2,"LONDON, July 19 - Solid-state batteries could...","[london, july, 19, solidstate, battery, could,..."
2,3,"STOCKHOLM/COPENHAGEN/OSLO, July 18 - SAS (SAS...","[stockholmcopenhagenoslo, july, 18, sa, sasst,..."
3,4,"WASHINGTON, July 18 - U.S. allies appear comm...","[washington, july, 18, u, ally, appear, commit..."
4,5,"FARNBOROUGH, England, July 18 - European airp...","[farnborough, england, july, 18, european, air..."
...,...,...,...
1476,1477,Dec 20 - The United States needs to do a bett...,"[dec, 20, united, state, need, better, job, ve..."
1477,1478,"BRUSSELS, Dec 20 - The European Commission pl...","[brussels, dec, 20, european, commission, plan..."
1478,1479,Dec 20 - Morgan Stanley (MS.N) on Monday made...,"[dec, 20, morgan, stanley, msn, monday, made, ..."
1479,1480,"BRUSSELS, Dec 20 - A European Union plan to c...","[brussels, dec, 20, european, union, plan, cha..."


In [ ]:
#create dictionary
dictionary = corpora.Dictionary(df['text_clean'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)

248487


In [ ]:
#create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['text_clean'] ]
print(len(doc_term_matrix))

1481


In [ ]:
lda = gensim.models.ldamodel.LdaModel

In [ ]:
num_topics=5
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

CPU times: user 2min 21s, sys: 1.1 s, total: 2min 22s
Wall time: 2min 23s


In [ ]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.013*"said" + 0.008*"carbon" + 0.007*"emission" + 0.006*"u" + 0.005*"company" + 0.005*"year" + 0.005*"reuters" + 0.005*"standard" + 0.004*"vehicle" + 0.004*"new"'),
 (1,
  '0.014*"said" + 0.009*"energy" + 0.008*"gas" + 0.008*"eu" + 0.007*"would" + 0.006*"country" + 0.006*"reuters" + 0.005*"standard" + 0.005*"european" + 0.005*"green"'),
 (2,
  '0.012*"said" + 0.005*"price" + 0.005*"worker" + 0.005*"reuters" + 0.004*"inflation" + 0.004*"year" + 0.004*"food" + 0.004*"read" + 0.003*"government" + 0.003*"would"'),
 (3,
  '0.016*"said" + 0.015*"company" + 0.008*"climate" + 0.006*"investor" + 0.006*"bank" + 0.005*"standard" + 0.005*"reuters" + 0.005*"fund" + 0.005*"year" + 0.005*"emission"'),
 (4,
  '0.014*"energy" + 0.013*"power" + 0.011*"said" + 0.008*"renewable" + 0.007*"wind" + 0.007*"solar" + 0.007*"year" + 0.007*"project" + 0.006*"hydrogen" + 0.005*"capacity"')]

In [ ]:
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

NameError: ignored

In [ ]:
# Assigns the topics to the documents in corpus
lda_corpus = ldamodel[doc_term_matrix]

In [ ]:
[doc for doc in lda_corpus]

[[(0, 0.23316573),
  (1, 0.09877049),
  (2, 0.36963743),
  (3, 0.29485264),
  (4, 0.0035737206)],
 [(0, 0.9957343),
  (1, 0.0010787075),
  (2, 0.0010577632),
  (3, 0.0010668786),
  (4, 0.0010623663)],
 [(0, 0.7354138),
  (1, 0.0008839131),
  (2, 0.26193357),
  (3, 0.00088891643),
  (4, 0.0008798106)],
 [(0, 0.4245737),
  (1, 0.051928904),
  (2, 0.00066019787),
  (3, 0.52218115),
  (4, 0.00065605744)],
 [(0, 0.99619937),
  (1, 0.00095122826),
  (2, 0.0009462294),
  (3, 0.0009552963),
  (4, 0.00094786927)],
 [(0, 0.00022912488),
  (1, 0.0005807169),
  (2, 0.077523574),
  (3, 0.5827301),
  (4, 0.33893642)],
 [(0, 0.0004468442),
  (1, 0.26365897),
  (2, 0.116508745),
  (3, 0.61893934),
  (4, 0.00044610963)],
 [(0, 0.32645765),
  (1, 0.0004921662),
  (2, 0.6720625),
  (3, 0.00049330754),
  (4, 0.0004943506)],
 [(0, 0.0010031394),
  (1, 0.0010045217),
  (2, 0.9959836),
  (3, 0.0010023081),
  (4, 0.001006461)],
 [(0, 0.0014497502),
  (1, 0.527149),
  (2, 0.468511),
  (3, 0.0014471062),
  (4, 

In [ ]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.20000000011182867


In [ ]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
# cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
# cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
# print(len(cluster4))
# print(len(cluster5))

407
592
309


In [ ]:
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel